In [7]:
%pip install polars
%pip install hvplot
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
   ---------------------------------------- 0.0/5.0 MB ? eta -:--:--
   ---------------------------------------- 5.0/5.0 MB 43.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 11.6/11.6 MB 60.7 MB/s eta

In [1]:
import polars as pl
import matplotlib.pyplot as plt

crew = pl.read_parquet("../tmp/crew.db.parquet/crew.parquet")

In [ ]:
# print(crew.describe())
print("\n".join(crew.columns))

In [ ]:
print(crew.head(3))

In [ ]:
print(crew.sample(3))

In [ ]:
rarity = crew.group_by("rarity").agg([
    pl.col("mint_offset").count().alias("count"),
    pl.col("agreeableness").mean(),
    pl.col("conscientiousness").mean(),
    pl.col("extraversion").mean(),
    pl.col("neuroticism").mean(),
    pl.col("openness").mean(),
]).sort("count", descending=True)

total_count = rarity["count"].sum()

rarity = rarity.with_columns((pl.col("count") / total_count * 100).alias("rarity %"))

print(rarity)

In [ ]:
# Commanders that are less agreeable and less neurotic
commanders = crew.filter([
    pl.col("rarity") != "common",
    pl.col("aptitudes").list.contains("command-major"),
    pl.col("agreeableness") < 0.1,
    pl.col("neuroticism") < 0.1,
    pl.col("extraversion") > 0.4,
])

# Top 10 commanders
out = commanders.with_columns((pl.col("agreeableness") + pl.col("neuroticism")).alias("scored")).sort("scored").limit(10)

print(out.select([
    pl.col("mint_offset"),
    pl.col("rarity"),
    pl.col("name"),
    pl.col("agreeableness"),
    pl.col("conscientiousness"),
    pl.col("extraversion"),
    pl.col("neuroticism"),
    pl.col("openness"),
]))

In [31]:
df = crew.select(pl.col("mint_offset"), pl.col("agreeableness")).sort("mint_offset")
print(df)
# plt.plot(df["mint_offset"], df["agreeableness"])

shape: (693_512, 2)
┌─────────────┬───────────────┐
│ mint_offset ┆ agreeableness │
│ ---         ┆ ---           │
│ i32         ┆ f64           │
╞═════════════╪═══════════════╡
│ 0           ┆ 0.8           │
│ 1           ┆ 0.38          │
│ 2           ┆ 0.74          │
│ 3           ┆ 0.12          │
│ 4           ┆ 0.99          │
│ …           ┆ …             │
│ 693507      ┆ 0.09          │
│ 693508      ┆ 0.58          │
│ 693509      ┆ 0.04          │
│ 693510      ┆ 0.2           │
│ 693511      ┆ 0.68          │
└─────────────┴───────────────┘
